## 01:-Experiment with gemini

In [46]:
# import os
# import google.generativeai as genai
# from dotenv import  load_dotenv
# # Load environment variables from .env file
# load_dotenv()
# api_key = os.getenv('GOOGLE_API_KEY')
# # Access your API key as an environment variable.
# genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
# # # Choose a model that's appropriate for your use case.
# model = genai.GenerativeModel('gemini-pro')

# prompt = "Write a story about cricket."

# response = model.generate_content(prompt)

# print(response.text)

- 1st: Method

In [33]:
import pandas as pd
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

In [30]:

# Sample PDF processing function
def get_pdf_text(pdf_paths):
    text = ""
    for pdf_path in pdf_paths:
        pdf_reader = PdfReader(pdf_path)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Sample text splitting function
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

    
def get_vector_store(text_chunks, api_key):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")
    return vector_store

def load_vector_store(api_key):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    return vector_store    



In [33]:
def get_conversational_chain(api_key):
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, google_api_key=api_key)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

def user_input(user_question, api_key):
    vector_store = load_vector_store(api_key)
    docs = vector_store.similarity_search(user_question)
    chain = get_conversational_chain(api_key)
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    print("Reply: ", response["output_text"])
    

In [35]:
# Simulate file processing
pdf_paths = ['constitution.pdf']
raw_text = get_pdf_text(pdf_paths)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks, api_key)

# Simulate user question and response generation
user_question = "What is the main topic discussed in the documents?"
user_input(user_question, api_key)


I0000 00:00:1722698406.916823 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722698411.458737 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722698413.603544 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722698413.604286 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Reply:      The main topic discussed in the documents is the Constitution of the United States and its amendments.


In [37]:
# Simulate user question and response generation
user_question = "What is the minimum age required to become a representative mentioned in section 2"
user_input(user_question, api_key)

I0000 00:00:1722703808.112220 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722703810.700452 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722703810.701926 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Reply:  25 years


In [41]:
# Simulate user question and response generation
user_question = "Explain the Article. II under section. 4?"
user_input(user_question, api_key)

I0000 00:00:1722704181.891682 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704184.142041 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704184.143086 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Reply:  This question cannot be answered from the given context.


In [42]:
# Simulate user question and response generation
user_question = "Explain the Article. III under section 4?"
user_input(user_question, api_key)

I0000 00:00:1722704194.582479 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704196.704248 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704196.704627 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Reply:  The provided text does not contain Article III, Section 4.


In [43]:
# Simulate user question and response generation
user_question = "How many articles are there?"
user_input(user_question, api_key)

I0000 00:00:1722704242.645950 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704244.848590 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704244.849593 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Reply:  7


In [45]:
# Simulate user question and response generation
user_question = "display all articles names?"
user_input(user_question, api_key)

I0000 00:00:1722704303.104485 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704305.268917 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722704305.269832 2972842 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Reply:  


- 2nd:Method

In [6]:
import pandas as pd
import fitz  # PyMuPDF
import docx
import json
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Load environment variables from .env file.
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

# Function to extract text from PDF, docx, and json files
def read_pdf(file_path):
    text = ""
    doc = fitz.open(file_path)
    for page in doc:
        text += page.get_text()
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def read_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return json.dumps(data)

def read_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            documents.append(read_pdf(file_path))
        elif filename.endswith('.docx'):
            documents.append(read_docx(file_path))
        elif filename.endswith('.json'):
            documents.append(read_json(file_path))
    return " ".join(documents)

# Sample text splitting function
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

    
def get_vector_store(text_chunks, api_key):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")
    return vector_store

def load_vector_store(api_key):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=api_key)
    vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
    return vector_store    

def get_conversational_chain(api_key):
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, google_api_key=api_key)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

def user_input(user_question, api_key):
    vector_store = load_vector_store(api_key)
    docs = vector_store.similarity_search(user_question)
    chain = get_conversational_chain(api_key)
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    print("Reply: ", response["output_text"])
    



In [7]:
# Simulate file processing
files_path = 'dataset/'
raw_text = read_documents(files_path)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks, api_key)

# Simulate user question and response generation
user_question = "What is the main topic discussed in the documents?"
user_input(user_question, api_key)

I0000 00:00:1722838009.433549  274574 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722838011.800852  274574 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722838014.159082  274574 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1722838014.161113  274574 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Reply:  Answer is not available in the context


### 02:- Experiments with opensource OpenAI Model

In [15]:
import os
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from dotenv import load_dotenv
import openai

# Load environment variables from .env file
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY_NEW')

# Function to extract text from PDF, docx, and json files

def read_pdf(file_path):
    text = ""
    doc = fitz.open(file_path)
    for page in doc:
        text += page.get_text()
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def read_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return json.dumps(data)

def read_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            documents.append(read_pdf(file_path))
        elif filename.endswith('.docx'):
            documents.append(read_docx(file_path))
        elif filename.endswith('.json'):
            documents.append(read_json(file_path))
    return " ".join(documents)

# Function to split text into chunks
def get_text_chunks(text):
    chunk_size = 10000
    chunk_overlap = 1000
    chunks = []
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# Function to create and save vector store
def get_vector_store(text_chunks):
    embeddings_model = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings_model)
    vector_store.save_local("faiss_index")
    return vector_store

# Function to load the vector store
def load_vector_store():
    embeddings_model = OpenAIEmbeddings()
    vector_store = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
    return vector_store

# Function to process user input and generate response using OpenAI model
def user_input(user_question):
    vector_store = load_vector_store()
    docs = vector_store.similarity_search(user_question)
    context = " ".join([doc.page_content for doc in docs])

    # Define a prompt template for the question-answering task
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer.\n\n
    Context:\n {context}\n
    Question: \n{question}\n

    Answer:
    """

    # Create the prompt with the given context and question
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    # Initialize the OpenAI LLM
    llm = OpenAI(api_key=openai.api_key, model="text-davinci-002") #gpt-4o-mini, text-davinci-002

    # Create the QA chain
    chain = LLMChain(
        llm=llm,
        prompt=prompt
    )

    # Get answer from the context
    result = chain.run(context=context, question=user_question)
    print("Reply:", result)



In [16]:
# Simulate file processing
files_path = 'dataset/'
raw_text = read_documents(files_path)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks, api_key)

# Simulate user question and response generation
user_question = "What is the main topic discussed in the documents?"
user_input(user_question, api_key)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## 03:- Experiment with opensource LLM Models ()

- 1st: Method

- meta-llama/Llama-2-7b-hf
- meta-llama/Llama-2-7b-chat-hf
- distilbert/distilbert-base-cased-distilled-squad
- google/gemma-2-2b
- Chetna19/m_albert_qa_model
- facebook/bart-large
- deepset/roberta-base-squad2
- facebook/bart-large-cnn

In [1]:
import os
import fitz
import docx
import json
from PyPDF2 import PdfReader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AutoModelForQuestionAnswering, pipeline
from dotenv import load_dotenv
import torch

# Load environment variables from .env file
load_dotenv()
api_key = os.getenv('HUGGINGFACE_API_KEY')


# Function to extract text from PDF, docx, and json files
def read_pdf(file_path):
    text = ""
    doc = fitz.open(file_path)
    for page in doc:
        text += page.get_text()
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

def read_json(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return json.dumps(data)

def read_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            documents.append(read_pdf(file_path))
        elif filename.endswith('.docx'):
            documents.append(read_docx(file_path))
        elif filename.endswith('.json'):
            documents.append(read_json(file_path))
    return " ".join(documents)

# Function to split text into chunks
def get_text_chunks(text):
    ## 1st method --------------
    splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = splitter.split_text(text)
    return chunks
    # ## 2nd method------------------------
    # chunk_size = 10000
    # chunk_overlap = 1000
    # chunks = []
    # for i in range(0, len(text), chunk_size - chunk_overlap):
    #     chunks.append(text[i:i + chunk_size])
    # return chunks
    

# Function to create and save vector store
def get_vector_store(text_chunks):
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings_model)
    vector_store.save_local("faiss_index")
    return vector_store

# Function to load the vector store
def load_vector_store():
    embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_store = FAISS.load_local("faiss_index", embeddings_model, allow_dangerous_deserialization=True)
    return vector_store

# Function to process user input and generate response using a model from Hugging Face
def user_input(user_question):
    vector_store = load_vector_store()
    docs = vector_store.similarity_search(user_question)
    context = " ".join([doc.page_content for doc in docs])

    #   Load the Hugging Face model and tokenizer
    #   meta-llama/Llama-2-7b-hf
    #   meta-llama/Llama-2-7b-chat-hf
    #   distilbert/distilbert-base-cased-distilled-squad
    #   bert-large-uncased-whole-word-masking-finetuned-squad
    #   google/gemma-2-2b
    #   Chetna19/m_albert_qa_model
    #   facebook/bart-large
    #   deepset/roberta-base-squad2
    #   facebook/bart-large-cnn
    #   ahotrod/electra_large_discriminator_squad2_512
    #   distilbert-base-uncased-distilled-squad
    #   t5-small, t5-large (770M parameters), t5-3b (3B parameters)
    
    model_name = "deepset/roberta-base-squad2"  # Replace with the correct model name
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=api_key)
    model = AutoModelForQuestionAnswering.from_pretrained(model_name, use_auth_token=api_key)
    # model = AutoModelForSeq2SeqLM.from_pretrained(model_name, use_auth_token=api_key)
    # model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=api_key)

    # Check if MPS (Metal Performance Shaders) is available, set the device
    device = 0 if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else -1

    # Create the question-answering pipeline
    qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer, device=device)
    # qa_pipeline = pipeline("question-answering", model="google/gemma-2-2b", device_map="auto", token=api_key)
    

    # Define a prompt template for the question-answering task
    # prompt_template = "Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
    provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    # Create the prompt with the given context and question
    prompt = prompt_template.format(context=context, question=user_question)

    # Process the context with the QA pipeline
    response = qa_pipeline(question=user_question, context=context)
    print("Reply:", response['answer'])


/Users/zeeshan/miniconda3/envs/LangChain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Simulate file processing
files_path = 'dataset'
raw_text = read_documents(files_path)
text_chunks = get_text_chunks(raw_text)
get_vector_store(text_chunks)

# Simulate user question and response generation
# user_question = "What is the main topic discussed in the documents?"
# user_question = "Who is Zeeshan?"
# user_question = "Who is Azam?"
# user_question = "Give name of the deepest and largest Ocean?"
# user_question = "Where is Rajasthan"
# user_question = "Give all famous list of cities in Rajasthan"
user_question = "Give capital city in Rajasthan"

user_input(user_question)

Reply: Jaipur
